## Get minsearch.py file and use it as package

In [1]:
from platform import python_version

print(python_version())

3.10.14


In [2]:
#!wget https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/main/01-intro/minsearch.py

In [3]:
import minsearch

## Get FAQ parser notebook

In [4]:
#!wget https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/main/01-intro/parse-faq.ipynb

## Get documents.json file generated after FAQ parsing

In [5]:
#!wget https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/main/01-intro/documents.json

In [6]:
import json

In [7]:
with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

In [8]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [9]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [10]:
index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

```keyword_fields``` operate like and SQL Query : ```SELECT * WHERE course = 'data-engineering-zoomcamp';```

In [11]:
q = 'the course has already started, can I still enroll?'

In [12]:
index.fit(documents)

```boost``` can be used when we think that one text field is more important than the other in the final result

In [13]:
boost = {
    'question': 3.0, # 3 times more important
    'section': 0.5, # 0.5 less important 
}

filter_d = {
    'course': 'data-engineering-zoomcamp'
}

index.search(
    query=q,
    filter_dict=filter_d,
    boost_dict=boost,
    num_results=5
)

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 202

In [14]:
import os
from mistralai.client import MistralClient
from mistralai.models.chat_completion import ChatMessage

In [15]:
api_key = os.environ["MISTRAL_API_KEY"]
model = "mistral-large-latest"

In [16]:
client = MistralClient(api_key=api_key)

chat_response = client.chat(
    model=model,
    messages=[ChatMessage(role="user", content=q)]
)

print(chat_response.choices[0].message.content)

That depends on the specific policy of the course provider. Some courses allow late enrollment, while others do not. I would recommend reaching out to the course provider directly to ask about their late enrollment policy. They can provide you with the most accurate information and may be able to offer alternatives if you are unable to enroll in the current session.


In [33]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5
    )

    return results

In [34]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [35]:
def llm(prompt):
    chat_response = client.chat(
        model=model,
        messages=[ChatMessage(role="user", content=prompt)]
    )
    
    return chat_response.choices[0].message.content

In [36]:
def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [44]:
query = "how to enroll in the course?"

In [45]:
rag(query)

"To enroll in the course, you need to register before the course starts using the registration link provided in the context. Additionally, it is recommended that you subscribe to the course public Google Calendar, join the course Telegram channel for announcements, and register in DataTalks.Club's Slack and join the channel. Even if you miss the start date, you can still join the course and submit homeworks, but be aware of the deadlines for turning in final projects."